<a href="https://colab.research.google.com/github/israelboco/42-Piscine-Unity/blob/master/TensorFlow_Recommenders.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Installling the TensorFlow Recommenders

In [1]:
!pip install -q tensorflow-recommenders
!pip install -q --upgrade tensorflow-datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.2/96.2 KB 4.3 MB/s eta 0:00:00


### Importing the libraries

In [2]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs

from typing import Dict, Text

### Reading the data

In [3]:
# Ratings data.
ratings = tfds.load('movielens/100k-ratings', split="train")
# Features of all the available movies.
movies = tfds.load('movielens/100k-movies', split="train")

# Select the basic features.
ratings = ratings.map(lambda x: {
    "movie_title": x["movie_title"],
    "user_id": x["user_id"]
})
movies = movies.map(lambda x: x["movie_title"])

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

...

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/movielens/100k-movies/0.1.1.incomplete5OCG8J/movielens-train.tfrecord*...:…

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


Dataset movielens downloaded and prepared to /root/tensorflow_datasets/movielens/100k-movies/0.1.1. Subsequent calls will reuse this data.


Build vocabularies to convert user ids and movie titles into integer indices for embedding layers:

In [5]:
user_ids_vocabulary = tf.keras.layers.StringLookup(mask_token=None)
user_ids_vocabulary.adapt(ratings.map(lambda x: x["user_id"]))

movie_titles_vocabulary = tf.keras.layers.StringLookup(mask_token=None)
movie_titles_vocabulary.adapt(movies)

###Define a model

We can define a TFRS model by inheriting from tfrs.Model and implementing the compute_loss method:

In [6]:
class MovieLensModel(tfrs.Model):

  def __init__(
      self,
      user_model: tf.keras.Model,
      movie_model: tf.keras.Model,
      task: tfrs.tasks.Retrieval):
    super().__init__()

    # Set up user and movie representations.
    self.user_model = user_model
    self.movie_model = movie_model

    # Set up a retrieval task.
    self.task = task

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    # Define how the loss is computed.

    user_embeddings = self.user_model(features["user_id"])
    movie_embeddings = self.movie_model(features["movie_title"])

    return self.task(user_embeddings, movie_embeddings)

Define the two Models and the retrieval task

In [7]:
# Define user and movie models.
user_model = tf.keras.Sequential([
    user_ids_vocabulary,
    tf.keras.layers.Embedding(user_ids_vocabulary.vocab_size(), 64)
])
movie_model = tf.keras.Sequential([
    movie_titles_vocabulary,
    tf.keras.layers.Embedding(movie_titles_vocabulary.vocab_size(), 64)
])

# Define your objectives.
task = tfrs.tasks.Retrieval(metrics=tfrs.metrics.FactorizedTopK(
    movies.batch(128).map(movie_model)
  )
)

### Fit and Evaluate

Creating the model,train it and generate predictions 

In [8]:
model = MovieLensModel(user_model,movie_model,task)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.5))

model.fit(ratings.batch(4096),epochs=3)

index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
index.index_from_dataset(
    movies.batch(100).map(lambda title: (title, model.movie_model(title))))



Epoch 1/3
25/25 [==============================] - 24s 762ms/step - factorized_top_k/top_1_categorical_accuracy: 8.0000e-05 - factorized_top_k/top_5_categorical_accuracy: 0.0015 - factorized_top_k/top_10_categorical_accuracy: 0.0050 - factorized_top_k/top_50_categorical_accuracy: 0.0453 - factorized_top_k/top_100_categorical_accuracy: 0.1020 - loss: 33085.6197 - regularization_loss: 0.0000e+00 - total_loss: 33085.6197
Epoch 2/3
25/25 [==============================] - 15s 599ms/step - factorized_top_k/top_1_categorical_accuracy: 1.0000e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0047 - factorized_top_k/top_10_categorical_accuracy: 0.0143 - factorized_top_k/top_50_categorical_accuracy: 0.1056 - factorized_top_k/top_100_categorical_accuracy: 0.2110 - loss: 31006.4684 - regularization_loss: 0.0000e+00 - total_loss: 31006.4684
Epoch 3/3
25/25 [==============================] - 16s 620ms/step - factorized_top_k/top_1_categorical_accuracy: 2.8000e-04 - factorized_top_k/top_5_categor

In [10]:
# Get some recommendations.
_, titles = index(np.array(["24"]))
print(f"Top 4 recommendations for user 24: {titles[0, :4]}")

Top 4 recommendations for user 24: [b'Waiting for Guffman (1996)' b'Party Girl (1995)'
 b"William Shakespeare's Romeo and Juliet (1996)" b'Jeffrey (1995)']
